# UX Notebook Reviwer

This notebook allows you to upload and evaluate other Jupyter Notebooks based on UX best practices. It checks for clarity, structure, readability, and completeness — helping notebook authors improve user experience.
The goal is to automate the review process using both structured logic and, eventually, an AI model.

### Usage Instructions:

0. **Install Required Packages:** What you will need. 
1. **Upload your Notebook:** Use the file uploader to select the `.ipynb` file you wish to review.
2. **Automatted UX Review:** Execute the code cells in sequence. The notebook will parse your uploaded file and apply the UX checklist.
3. **Review the Report:** A markdown table will be generated, detailing the status of each guideline and providing suggestions where applicable.
4. **Add Custom Notes (Optional):** Utilize the "Reviewer Notes" section to add any additional qualitative feedback or summary.
5. **Export (Optional):** Download or copy the generated review markdown for easy sharing.

---

## 0. Install Required Pacakges
Before running any of the automation or parsing cells, make sure you have these core dependencies:

- **ipywidgets** (≥7.6.0): for interactive loading indicators  
- **nbformat**   (≥5.1.3): to parse `.ipynb` files into Python objects  
- **openai**     (≥0.27.0): your Ollama-shim client library  

In [ ]:
# Install required packages (run once)
!pip install --upgrade ipywidgets nbformat openai pandas matplotlib

---

## 1. Upload Notebook File
Creates an upload button where you can select a .ipynb file. Once uploaded, the file will be held in memory and ready for review.

In [42]:
from ipywidgets import FileUpload
from IPython.display import display

upload = FileUpload(accept='.ipynb', multiple=False)
display(upload)

FileUpload(value=(), accept='.ipynb', description='Upload')

---

## 2. Automated UX Review (LLM-Powered)

This section uses a local LLM (via Ollama) to evaluate the structure and user experience of your uploaded Jupyter Notebook.  
It applies a structured checklist based on best practices for clarity, reusability, and reader guidance.
1. **Extract Notebook Content**: Gathers all markdown and code cells into a plain-text string.
2. **Build Structured Prompt**: Injects a legend (✅/🔹/❌/–) and the exact table skeleton so the model knows what to fill in.
3. **Invoke LLM**: Sends the prompt to your local Ollama-backed model via the OpenAI shim.
4. **Parse & Render**: Filters the response to display only the filled-in Markdown table with left-aligned suggestions.

By embedding this automated review step, you get a consistent, reproducible UX evaluation right inside your notebook—no manual scoring required. 

In [ ]:
import nbformat

def load_notebook(upload_widget):
    if not upload_widget.value or len(upload_widget.value) == 0:
        return None, "❌ No file uploaded."

    uploaded = upload_widget.value

    # Handle both tuple and dict formats
    if isinstance(uploaded, tuple) and len(uploaded) > 0 and isinstance(uploaded[0], dict):
        file_info = uploaded[0]
        filename = file_info.get("name", "Unnamed Notebook")
        content = file_info["content"]
    elif isinstance(uploaded, dict):
        filename, file_info = next(iter(uploaded.items()))
        content = file_info["content"]
    else:
        return None, "❌ Unexpected upload structure."

    try:
        nb = nbformat.reads(bytes(content).decode('utf-8'), as_version=4)
        return nb, filename
    except Exception as e:
        return None, f"❌ Error reading notebook: {e}"

# Load the notebook
notebook_data, notebook_name = load_notebook(upload)

# Feedback
if notebook_data:
    print(f"✅ Loaded notebook: {notebook_name} with {len(notebook_data.cells)} cells.")
else:
    print(notebook_name)


---

In [44]:
# ───── Cell: UX Notebook Review w/ Legend ─────
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# 1) Extract your notebook’s text
notebook_text = extract_notebook_text(notebook_data)

# 2) Build prompt with legend + strict, left-aligned table skeleton
review_prompt = f"""
Here is a Jupyter notebook for review:

{notebook_text}

**Legend (think of each icon as a %-based score):**  
✅ Green check: 90–100% compliance (excellent)  
🔹 Blue diamond: 60–89% compliance (good but needs a little work)  
❌ Red X: 0–59% compliance or completely missing (major issue)  
– Hyphen: not applicable  

You MUST return *only* a GitHub-flavored Markdown table with these columns, 
in this exact order, and no other text:

| Guideline              | Status | Suggestion                                        |
|------------------------|--------|:--------------------------------------------------|
| Clear Header           |        |                                                   |
| Goal/Objective Present |        |                                                   |
| Setup & Pre-Requisites |        |                                                   |
| Markdown Usage         |        |                                                   |
| Avoid Hardcoding       |        |                                                   |
| Inline Code Comments   |        |                                                   |
| Next Steps & Conclusion|        |                                                   |

Fill in each row’s **Status** (using the legend above) and **Suggestion**.  
Use ❌ if the notebook does **not** address this guideline at all.
"""

# 3) Send to your Ollama-backed LLM
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
loading = widgets.HTML("<b>🧠 Evaluating the notebook…</b>")
display(loading)

response = client.chat.completions.create(
    model="llama3.1:8b-instruct-fp16",
    messages=[
        {"role": "system", "content": "You are a helpful UX reviewer of technical Jupyter Notebooks."},
        {"role": "user",   "content": review_prompt},
    ],
    temperature=0.3,
)

# 4) Filter output: keep legend lines plus the table
clear_output()
raw = response.choices[0].message.content.splitlines()

legend_block = []
table_block  = []
in_table = False
for ln in raw:
    if ln.strip().startswith("|"):
        in_table = True
        table_block.append(ln)
    elif not in_table and ln.strip():
        legend_block.append(ln)

# 5) Render legend + table
output_md = "\n".join(legend_block + [""] + table_block)
display(Markdown(output_md))

Here is a review of your Jupyter Notebook based on the guidelines you provided:

| Guideline              | Status | Suggestion                                        |
|------------------------|--------|:--------------------------------------------------|
| Clear Header           | ✅      | Excellent header with clear title and sections   |
| Goal/Objective Present | 🔹      | Consider adding a brief summary or objective at the beginning of the notebook. This will help users understand what they can expect from running this notebook. |
| Setup & Pre-Requisites | 🔹      | The setup section is mostly complete, but consider adding more details about the required dependencies and environment settings. For example, you could mention that Python 3.11 is recommended and that later versions are not supported. |
| Markdown Usage         | ✅      | Excellent use of markdown throughout the notebook, with clear headings, paragraphs, and code blocks. |
| Avoid Hardcoding       | 🔹      | There are several instances where hardcoded values are used (e.g., API keys, model IDs). Consider using environment variables or configuration files to make these values more flexible and easier to manage. |
| Inline Code Comments   | ✅      | Excellent use of inline comments to explain code logic and provide context. |
| Next Steps & Conclusion| 🔹      | The notebook concludes with a summary of the steps taken, but consider adding more details about what users can expect next (e.g., running the resulting `seed.jsonl` file in another notebook). |

# Next Steps
- Should we create suggestions?
- What should users do with this information?
- export?
- github integration
- timer
- download new notebook